# Test OpenAPI Specification Generation

This notebook tests the new OpenAPI specification generation functionality added to the notebook-nanoservice project.

In [ ]:
from notebook_nanoservice import NanoService
import json

# Define some test functions with various parameter types
def simple_function(name: str):
    """A simple function that greets someone."""
    return f"Hello, {name}!"

def complex_function(age: int, height: float, is_student: bool, hobbies: list[str]):
    """A function with multiple parameter types."""
    return {
        "age": age,
        "height": height,
        "is_student": is_student,
        "hobbies": hobbies
    }

def create_image(size: int):
    """Creates a simple image - should be detected as image response."""
    from PIL import Image
    return Image.new('RGB', (size, size), color='blue')

def no_params():
    """Function with no parameters."""
    return "No parameters needed!"

print("Test functions defined!")

In [ ]:
# Start the nanoservice
service = NanoService(globals(), port=5002)
service.start()
print("Service started on http://localhost:5002")

In [ ]:
# Test the standard JSON API with the new params field
import requests

response = requests.get("http://localhost:5002")
api_data = response.json()

print("=== Standard JSON API Response ===")
print(f"Status: {response.status_code}")
print("\nFunctions found:")
for func_name in api_data['api'].keys():
    print(f"- {func_name}")

print("\n=== Sample Function Info ===")
sample_func = 'complex_function'
if sample_func in api_data['api']:
    func_info = api_data['api'][sample_func]
    print(f"Function: {sample_func}")
    print(f"Signature: {func_info['signature']}")
    print(f"Doc: {func_info['doc']}")
    print(f"Params: {json.dumps(func_info['params'], indent=2)}")
else:
    print(f"Function {sample_func} not found")

In [ ]:
# Test the new OpenAPI specification generation
response = requests.get("http://localhost:5002?format=openapi")
openapi_spec = response.json()

print("=== OpenAPI Specification ===")
print(f"Status: {response.status_code}")
print(f"OpenAPI Version: {openapi_spec.get('openapi')}")
print(f"API Title: {openapi_spec.get('info', {}).get('title')}")
print(f"API Version: {openapi_spec.get('info', {}).get('version')}")

print("\n=== Available Paths ===")
for path in openapi_spec.get('paths', {}).keys():
    print(f"- {path}")

print("\n=== Sample Path Details ===")
sample_path = '/api/complex_function'
if sample_path in openapi_spec.get('paths', {}):
    path_info = openapi_spec['paths'][sample_path]
    print(f"Path: {sample_path}")
    print(f"Methods: {list(path_info.keys())}")
    
    if 'get' in path_info:
        get_op = path_info['get']
        print(f"\nGET Operation:")
        print(f"  Summary: {get_op.get('summary')}")
        print(f"  Parameters: {len(get_op.get('parameters', []))}")
        for param in get_op.get('parameters', []):
            print(f"    - {param['name']} ({param['schema']['type']}, required: {param['required']})")
    
    if 'post' in path_info:
        post_op = path_info['post']
        print(f"\nPOST Operation:")
        print(f"  Summary: {post_op.get('summary')}")
        if 'requestBody' in post_op:
            schema = post_op['requestBody']['content']['application/json']['schema']
            print(f"  Request Body Properties: {list(schema.get('properties', {}).keys())}")
            print(f"  Required Fields: {schema.get('required', [])}")
else:
    print(f"Path {sample_path} not found")

In [ ]:
# Test parameter extraction for different function types
print("=== Parameter Extraction Test ===")

test_functions = ['simple_function', 'complex_function', 'create_image', 'no_params']

for func_name in test_functions:
    if func_name in api_data['api']:
        params = api_data['api'][func_name]['params']
        print(f"\n{func_name}:")
        if params:
            for param in params:
                type_info = param['type']
                if param['type'] == 'array':
                    type_info += f" of {param.get('items', {}).get('type', 'unknown')}"
                print(f"  - {param['name']}: {type_info} (required: {param['required']})")
        else:
            print("  No parameters")
    else:
        print(f"\n{func_name}: Not found")

In [ ]:
# Pretty print the full OpenAPI spec for inspection
print("=== Full OpenAPI Specification ===")
print(json.dumps(openapi_spec, indent=2))

In [ ]:
# Test that the existing API functionality still works
print("=== Testing Existing API Functionality ===")

# Test simple function
response = requests.get("http://localhost:5002/api/simple_function?name=OpenAPI")
print(f"Simple function test: {response.status_code} - {response.text}")

# Test complex function via POST
payload = {
    "age": 25,
    "height": 5.8,
    "is_student": True,
    "hobbies": ["reading", "coding"]
}
response = requests.post("http://localhost:5002/api/complex_function", json=payload)
print(f"Complex function test: {response.status_code} - {response.text}")

# Test no params function
response = requests.get("http://localhost:5002/api/no_params")
print(f"No params function test: {response.status_code} - {response.text}")

In [ ]:
# Stop the service
service.stop()
print("Service stopped.")